# Build a Text Retrieve Example with TiDB, Amazon Titan Text Embeddings V2, Amazon Bedrock and Boto3

## Introduction

In this notebook we will show you how to use TiDB and Boto3 SDK to retrieve text with Amazon Titan Text Embeddings V2 and Amazon Bedrock. [TiDB](https://tidb.cloud/?utm_source=github&utm_medium=community&utm_campaign=video_aws_example_generativeai_cm_0624) is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads. It is MySQL compatible and features horizontal scalability, strong consistency, and high availability. You can deploy TiDB in a self-hosted environment or in the cloud.

### Use case

To demonstrate the vector search capability of TiDB Serverless, and embedding capability of Amazon Titan Text Embeddings V2, let's take the use case of vector cosine distance search.

### Persona

You are a TiDB user, you want to build a searching engine about TiDB. So you need to retrieve the related text to the question, using embedding and vector search features to achieve it.

### Implementation

To fulfill this use case, in this notebook we will show how to create embedding vectors based on the text chunks, save it to TiDB Serverless, use vector search feature to get the nearest vector and its text chunk. We will use the TiDB and the Amazon Titan Text Embeddings V2 model through the Amazon Bedrock API with Boto3 SDK.

### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠

## Installation

To run this notebook you would need to install dependencies - SQLAlchemy, tidb-vector, PyMySQL, boto3.

In [ ]:
%%capture
%pip install --upgrade pip
%pip install PyMySQL==1.1.0 --force-reinstall --quiet
%pip install SQLAlchemy==2.0.30 --force-reinstall --quiet
%pip install tidb-vector==0.0.9 --force-reinstall --quiet
%pip install boto3 --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup

Import the necessary libraries

In [ ]:
from sqlalchemy import Column, Integer, Text, create_engine, URL
from sqlalchemy.orm import Session, declarative_base
from tidb_vector.sqlalchemy import VectorType

import json
import os
import boto3

## Initialization

Connect to a TiDB Cloud Cluster and Initiate Bedrock Runtime Client

In [ ]:
def get_db_url():
    return URL(
        drivername="mysql+pymysql",
        username=os.environ["TIDB_USER"],
        password=os.environ["TIDB_PASSWORD"],
        host=os.environ['TIDB_HOST'],
        port=int(os.environ["TIDB_PORT"]),
        database=os.environ["TIDB_DB_NAME"],
        query={"ssl_verify_cert": True, "ssl_verify_identity": True},
    )

engine = create_engine(get_db_url(), pool_recycle=300)
bedrock_runtime = boto3.client('bedrock-runtime')

## Payload Creation

Create payload to request Amazon Titan Text Embeddings V2 to make an embedding vector。

In [ ]:
model_name = "amazon.titan-embed-text-v2:0"
dim_of_embedding_model = 512

def create_payload(input_text: str):
    return {
        "modelId": model_name,
        "contentType": "application/json",
        "accept": "*/*",
        "body": {
            "inputText": input_text,
            "dimensions": dim_of_embedding_model,
            "normalize": True,
        }
    }

## TiDB Table and Vector Index

Create table and its vector index in TiDB Serverless to store text and vector.

In [ ]:
Base = declarative_base()
class Entity(Base):
    __tablename__ = "entity"

    id = Column(Integer, primary_key=True)
    content = Column(Text)
    content_vec = Column(
        VectorType(dim=dim_of_embedding_model),
        comment="hnsw(distance=l2)"
    )

Base.metadata.create_all(engine)

## Model invocation

Invoke the Amazon Titan Text Embeddings V2 model using bedrock runtime client

Amazon Bedrock runtime client provides you with an API `invoke_model` which accepts the following:
- `modelId`: This is the model ARN for the foundation model available in Amazon Bedrock
- `accept`: The type of input request
- `contentType`: The content type of the output
- `body`: A json string payload consisting of the prompt and the configurations

In [ ]:
def embedding(content):

    payload = create_payload(content)
    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')

    # Invoke the model
    response = bedrock_runtime.invoke_model(
        body=body_bytes,
        contentType=payload['contentType'],
        accept=payload['accept'],
        modelId=payload['modelId']
    )

    result_body = json.loads(response.get("body").read())

    return result_body.get("embedding")

## Save Vector to TiDB Serverless

Save 3 records with embedding vector to TiDB Serverless.

In [ ]:
tidb_content = 'TiDB is an open-source distributed SQL database that supports \
Hybrid Transactional and Analytical Processing (HTAP) workloads.'
tikv_content = 'TiKV is an open-source, distributed, and transactional \
key-value database. Unlike other traditional NoSQL systems.'
pd_content = 'The Placement Driver (PD) server is the metadata \
managing component of the entire cluster.'

with Session(engine) as session:
    session.add(Entity(content = tidb_content, content_vec = embedding(tidb_content)))
    session.add(Entity(content = tikv_content, content_vec = embedding(tikv_content)))
    session.add(Entity(content = pd_content, content_vec = embedding(pd_content)))
    session.commit()

# Retrieve Content via Vector Cosine Distance

In [ ]:
query = 'What is TiDB?'
embedding_query = embedding(query)

with Session(engine) as session:
    entity = session.query(Entity).order_by(
        Entity.content_vec.cosine_distance(embedding_query)
    ).limit(1).first()

    print(entity.content)